In [ ]:
# only to mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Installing necesarry packages

! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece] scikit-learn

     |████████████████████████████████| 311 kB 8.2 MB/s 
     |████████████████████████████████| 3.5 MB 54.8 MB/s 
     |████████████████████████████████| 90 kB 13.0 MB/s 
     |████████████████████████████████| 1.2 MB 52.2 MB/s 
     |████████████████████████████████| 212 kB 67.4 MB/s 
     |████████████████████████████████| 1.1 MB 52.7 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
     |████████████████████████████████| 134 kB 75.5 MB/s 
     |████████████████████████████████| 895 kB 54.6 MB/s 
     |████████████████████████████████| 6.8 MB 38.3 MB/s 
     |████████████████████████████████| 596 kB 70.0 MB/s 
     |████████████████████████████████| 271 kB 77.2 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 57.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, MarianMTModel, MarianTokenizer

from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split
import datasets

import random
import numpy as np
import pandas as pd
from IPython.display import display, HTML


os.environ["WANDB_DISABLED"]="true"
print(transformers.__version__)

4.16.2


In [ ]:
# All necessary variables
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "zu"
batch_size = 8
epoch = 1
learning_rate=1e-5
weight_decay = 0.001
model_checkpoint = "/content/drive/MyDrive/Africa-NLP-Paper/opus-mt-en-fr-finetuned-en-to-zu/checkpoint-5500" # Path to the saved fine-tuned model.
model_name = model_checkpoint.split("/")[-1]

In [ ]:
# Uploading data
base_url = '/content/drive/MyDrive/Africa-NLP-Paper/'
raw_datasets = load_dataset('csv', data_files=[base_url + 'en-zu.training.csv'])
test_dataset = load_dataset('csv', data_files=[base_url + 'en-zu.eval.csv'])
# Preparing data
test_dataset = test_dataset.remove_columns('zu')
test_dataset = test_dataset.rename_column('Source','source')
test_dataset = test_dataset.rename_column('zu.1','zu')
# Uploading Metrics
metric = load_metric("sacrebleu")

Using custom data configuration default-73c6d49c79903342


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-73c6d49c79903342/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-889bf343aa5166a9/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

In [ ]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(test_dataset["train"])

,en,zu,source
0,"They are very poor people and they lose all they have,"" the Afghan Red Crescent Society's secretary general Nilab Mobarez said.","Bangabantu abadla imbuya ngothi futhi balahlekelwa yiko konke abanakho,” kusho unobhala-jikelele we-Afghan Red Crescent Society, u-Nilab Mobarez.",News Crawl 2019
1,"It's the anti-gender binary, to be immersed in seeing the world with a lot of different possibilities through stories.","Kuwukuphikisana nokuhlukaniswa kwabantu ngobulili obubhekwa njengobuhleliwe, ukwamukela ukubheka izwe njengelinamathuba amaningi ahlukahlukene ngezindaba ezixoxwayo.",News Crawl 2019
2,"Tell that story through music, specifically hip-hop, rap and some exquisite R&B ballads that wouldn't sound out of place being performed by Childish Gambino or Beyoncé and you are starting to understand why Hamilton has gripped the world.","Xoxa indaba ngomculo, ikakhulukazi nge-hip-hop, i-rap kanye nomculo opholile omnandi kakhulu we-R&B ongeke uzwakale ungangeni lapho uculwa uGambino noma uBeyoncé futhi uqala ukuqonda ukuthi kungani uHamilton ethandwe kakhulu umhlaba.",News Crawl 2019
3,"""It's done, we move forward,"" Raheem Sterling wrote on Instagram in the early hours of Tuesday, in the wake of his confrontation with Liverpool's Joe Gomez at St George's Park on Monday.","“Sesiqedile, siqhubekela phambili,” kubhala u-Raheem Sterling ku-Instagram ekuseni ngoLwesibili, ngemva kokuhilizisana no-Joe Gomez we-Liverpool e-St George’s Park ngoMsombuluko.",News Crawl 2019
4,Morgan was featured on Time magazine's cover.,UMorgan wayevezwe ekhasini elingaphambili loMagazine iTime,News Crawl 2019


In [ ]:

def preprocess_function(examples):
    '''
    Pre- and post-processing of tokens
    '''
    inputs = [prefix + ex for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def postprocess_text(preds, labels):
    '''
    post processing the outputs for BLEU
    '''
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
    
def compute_metrics(eval_preds):
    '''
    computes BLEU
    '''
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) 
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)



  0%|          | 0/1 [00:00<?, ?ba/s]

OSError: ignored

In [ ]:
tokenized_train_dataset = raw_datasets.map(preprocess_function, batched=True)

In [ ]:
args = Seq2SeqTrainingArguments(
    f"{base_url}{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_total_limit=3,
    num_train_epochs=epoch,
    predict_with_generate=True    
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # initializing data collator

In [ ]:
# Training args
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset["train"],
    eval_dataset=tokenized_test_dataset["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate(tokenized_test_dataset['train'])